In [0]:
# import tkinter as tk                # python 3 #tkinter for the GUI: graphical interface
from tkinter import font  as tkfont # python 3 
#import Tkinter as tk     # python 2
#import tkFont as tkfont  # python 2
import imutils #image processing
import glob  #for env path
import numpy as np 
import functions as f 
import core as c
import pandas as pd
from PIL import Image, ImageTk
#import DetectCountry
#import extractLicense
from tkinter import ttk as ttk
import cv2
#import Main
import json
import csv
from os import environ
import os
import datetime
import petl as etl
from tkinter import *
from PIL import Image, ImageTk
import time
# this example leverages the requests module
# it can be installed using pip install requests
try:
    import requests
except ImportError:
    print("[ERROR] please install the requests module")
    raise

from requests.auth import HTTPDigestAuth
alpr_dir=r'openalpr\openalpr_64'
environ['PATH'] = alpr_dir + ';' + environ['PATH']

from openalpr import Alpr #automatic license plate recognition  library 
alpr = Alpr('eu', alpr_dir + '/openalpr.conf', alpr_dir + '/runtime_data')

if not alpr.is_loaded():
    print("Error loading OpenALPR")
    sys.exit(1)
    
alpr.set_top_n(10)
alpr.set_default_region("tn")
support=""
class SampleApp(tk.Tk):

    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)
        #fixing the window name on the wwindow top bar
        tk.Tk.wm_title(self,"welcome to AVENGUARD")
        #fixing the app icon using windows
        icon = "To_ico.ico"
        tk.Tk.iconbitmap(self, icon)
        #defining the defaut text features created under the window
        self.title_font = tkfont.Font(family='Helvetica', size=18, weight="bold", slant="italic")
        #self.resizable(width=False, height=False)

        # the container is where we'll stack a bunch of frames
        # on top of each other, then the one we want visible
        # will be raised above the others
        container = tk.Frame(self)
        container.pack(side="top", fill="both")
        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)
        
        self.frames = {}
        for F in (StartPage, PageOne, PageTwo, PageHelp):
            page_name = F.__name__
            frame = F(parent=container, controller=self)
            self.frames[page_name] = frame

            # put all of the pages in the same location;
            # the one on the top of the stacking order
            # will be the one that is visible.
            frame.grid(row=0, column=0, sticky="nsew")

        self.show_frame("StartPage")

    def show_frame(self, page_name):
        '''Show a frame for the given page name'''
        frame = self.frames[page_name]
        frame.tkraise()


class StartPage(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller
        im="logo_avensys.png"
        logo = ImageTk.PhotoImage(file=im)
        w1 = tk.Label(self, image=logo)
        w1.image=logo #we have to keep a refrence otherwise the image will be transparent totally transparet!!
        w1.pack(side="top")
        
        label1 = tk.Label(self, text="Welcome to the ALPR solution developped By Avensys", font=controller.title_font)
        label1.pack(side="top", fill="x", pady=7)
        label2 = tk.Label(self, text="Avensys... your security provider", font=controller.title_font)
        label2.pack(side="top", fill="x", pady=7)
    

class PageOne(tk.Frame):

            
    def __init__(self, parent, controller):
        
        def repondre():
            affichage['text'] = reponse.get()	# lecture du contenu du widget "reponse"
            file = open(r'C:\avenguard\files\IPAddress.txt','a') 
            support=affichage['text']
            file.write(affichage['text']+'\n') 
            file.close()
            return support
            
        
        tk.Frame.__init__(self, parent)
        self.controller = controller
        
        button = ttk.Button(self, text="Go to the start page",
                           command=lambda: controller.show_frame("StartPage"))
        button.pack(side="bottom")
        
        nom = tk.Label(self, text = 'Enter the ip camera as follow: user:pwd@ip_addr:port_number please :')
        reponse = tk.Entry(self)
        valeur = ttk.Button(self, text ='confirm',command=repondre)
        affichage = tk.Label(self)
        votre_add=tk.Label(self, text='You have selected successfully the following IP address :')
        nom.pack(expand=True)
        reponse.pack(expand=True)
        valeur.pack(expand=True)
        votre_add.pack(expand=True)
        affichage.pack(expand=True)    
        
class PageTwo(tk.Frame):

    def __init__(self, parent, controller):
        
        
        # variables
        fileHandle = open ( r'C:\avenguard\files\IPAddress.txt',"r" )
        lineList = fileHandle.readlines()
        fileHandle.close()
        s=lineList[-1]
        url=''
        dir="C:/avenguard/temporary_images"
        i=0 
        for char in s:
            if (i<len(s)-1):
                url=url+char
                i=i+1
        USER=url.split(':')[0]
        PASS=url.split(':')[1].split('@')[0]
        HOSTNAME=url.split('@')[1]


        # build the HTTP digest auth
        #DIGEST = HTTPDigestAuth(username=USER, password=PASS)
        #URL =  HOSTNAME + "/axis-cgi/jpg/image.cgi"
            
        def show_stream():
            try:
                URL =  HOSTNAME + "/axis-cgi/jpg/image.cgi"
                original=f.download_image(URL,USER,PASS, "image.jpeg")
            except: #ConnectionError:
                URL2 = "http://"+HOSTNAME + "/axis-cgi/jpg/image.cgi"
                original=f.download_image(URL2, USER,PASS,"image.jpeg")
            #except:
                #original="image.jpeg"
                #print("we are here")
            v2=Image.open(original).resize((500,300), Image.ANTIALIAS)
            logo2 = ImageTk.PhotoImage(v2)
                
            w3 = tk.Label(self, image=logo2)
            w3.image=logo2 #we have to keep a refrence otherwise the image will be transparent totally transparet!!
            w3.grid(row = 0, column = 0, rowspan=4,columnspan=4,sticky="nswe",padx = 10) #,sticky="nswe"
            self.after(10,lambda:show_stream())
        
        def show2():
            import dfgui
            import pandas as pd 
            data = pd.read_csv(r'C:\avenguard\files\results.csv') 
            w = Canvas(self,dfgui.show(data), width=1000, height=1000)
            w.pack()
            self.after(100,lambda:show2())
        def show():
            
            # open file
            #try to open the file
            #cropped=r"C:/avenguard/stored_plates/cropped.jpg"
            #v_camera = Image.open(cropped).resize((500,300), Image.ANTIALIAS)
            #logo = ImageTk.PhotoImage(v_camera)
           # w1 = tk.Label(self, image=logo)
            #w1.image=logo #we have to keep a refrence otherwise the image will be transparent totally transparet!!
            #w1.grid(row = 0, column = 0, rowspan=4,columnspan=6,sticky="w",padx = 10)
            try:
                
                #Load the table of the results  
                table1 = etl.fromcsv(r'C:\avenguard\files\results.csv')
                table2 = etl.rowlenselect(table1, 12)
                # Alter the colums
                table2 = etl.cut(table2, 'plate','timestamp','region')
                table2 = etl.split(table2, 'timestamp', '_', ['date', 'heure'])
                table2 = etl.tail(table2, 15) # the last 15 lines of detcted licenses 
              
                # Save to new temporary file
                etl.tocsv(table2, r'C:\avenguard\files\temp.csv')
                
                with open(r"C:\avenguard\files\temp.csv","r") as file:
                    
                    reader = csv.reader(file)
            
                    # r and c tell us where to grid the labels
                    r = 5
                    #list=[r"C:/avenguard/callable_plates/IG75038.jpg"]
                    list=[r"C:/avenguard/callable_plates/77 TU 5232.jpg"]
                    for col in reader:
                        c = 0
                      
                        for row in col:
                            
                            if r==5 :
                                if row=='plate':
                                    
                                    # i've added some styling
                                    label = tk.Label(self, width = 24, height = 2,text = 'plaque detectée', borderwidth=0,fg='blue')
                                    label.grid(row = r, column = c,rowspan=1)
                                    label2 = tk.Label(self, width = 14, height = 2,text = 'numero detecté', borderwidth=0,fg='blue')
                                    label2.grid(row = r, column = c+1,rowspan=1)
                                elif row=="timestamp":
                                    label = tk.Label(self, width = 16, height = 2,text = 'date de capture',  borderwidth=0,fg='blue')
                                    label.grid(row = r, column = c+1,rowspan=1)
                                else:
                                    label = tk.Label(self, width = 14, height = 2,text = row,  borderwidth=0,fg='blue')
                                    label.grid(row = r, column = c+1,rowspan=1)
                            
                            elif c==0 :
                                 
                                 # i've added some styling
                                
                                img1=r"C:/avenguard/callable_plates/"+row+".jpg"
                                img2=list[-1]
                                same=f.compare_plates(img1,img2)
                                #print (same)
                                list.append(img1)
                                v=Image.open(img1).resize((166,35),Image.ANTIALIAS)
                                logo=ImageTk.PhotoImage(v)
                                
                                label = tk.Label(self, image=logo, text=row)
                                label.image=logo
                                def openimage(event):
                                    
                                        mylabel=event.widget #this line is so important that i spent a whole morning 
                                                          #working on it otherwise the function will show only the last image
                                        image=r"C:/avenguard/callable_plates/"+mylabel.cget("text")+".jpg"
                                        im=Image.open(image)
                                        im.show()
                                       
                                label.grid(row = r, column = c,rowspan=1)
                                label.bind("<Button-1>", openimage)
                                label2= tk.Label(self, width = 14, height = 2,text = f.insert_type(row),  borderwidth=0)
                                label2.grid(row = r, column = c+1,rowspan=1)
                            elif c==1 :
                                label = tk.Label(self, width = 16, height = 2,text = row.replace("-",":"),  borderwidth=0)
                                label.grid(row = r, column = c+1,rowspan=1)
                            else  :
                                # i've added some styling
                                label = tk.Label(self, width = 14, height = 2,text = row, borderwidth=0)
                                label.grid(row = r, column = c+1,rowspan=1)
                            c += 1
                        r += 1
                     
            except IOError as e:
                print (e)
            else:
                
                self.after(10,lambda:show())  #to update the content of the GUI 
        
        def execute():
          
           # EXTRACODE for Tunisian Licence Type 
                def getLicenceType(template,imagePath): 
                    # load the image, convert it to grayscale, and initialize the
                    # bookkeeping variable to keep track of the matched region
                    image = cv2.imread(imagePath)
                    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

                    found = None
 
                    # loop over the scales of the image
                    for scale in np.linspace(0.2, 1.0, 20)[::-1]:
                        # resize the image according to the scale, and keep track
                        # of the ratio of the resizing
                        resized = imutils.resize(gray, width = int(gray.shape[1] * scale))
                        r = gray.shape[1] / float(resized.shape[1])
 
                        # if the resized image is smaller than the template, then break
                        # from the loop
                        if resized.shape[0] < tH or resized.shape[1] < tW:
                            break
                        # detect edges in the resized, grayscale image and apply template
                        # matching to find the template in the image
                        edged = cv2.Canny(resized, 50, 200)
                        result = cv2.matchTemplate(edged, template, cv2.TM_CCOEFF)
                        (_, maxVal, _, maxLoc) = cv2.minMaxLoc(result)
 
                        # if we have found a new maximum correlation value, then ipdate
                        # the bookkeeping variable
                        if found is None or maxVal > found[0]:
                            found = (maxVal, maxLoc, r)

                    # unpack the bookkeeping varaible and compute the (x, y) coordinates
                    # of the bounding box based on the resized ratio
                    if found:
                        return template_source.split('\\')[-1].split('.')[1]
                
                
                
                now=datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S") #reading actual time of capture =timestamp format 
                snow=str(now)
                nows= "C:/avenguard/temporary_images/" +snow+".jpg"  #convert timestamp into string for compatibility reason
                #print (nows)
                # build the HTTP digest auth
                DIGEST = HTTPDigestAuth(username=USER, password=PASS)
                URL = "http://"+HOSTNAME + "/axis-cgi/jpg/image.cgi"
                #print (URL)
                try:
                    output_file = f.download_image(URL, nows)
                except:
                    URL2=HOSTNAME + "/axis-cgi/jpg/image.cgi"
                    output_file = f.download_image(URL2, nows)
                car_model='test'
                results={}
                try:
                  # start of recognition algorithm code 
                    results = alpr.recognize_file(nows)
                except:
                    print('Error recongnizing the image')
                
                try:
                    if results:
                        t=results['results'][0]
                        t[u'timestamp']=snow
                        t[u'car_model']=car_model 
                        res_data = results['results']
                except:
                    res_data = results['results']
                
                #cropped="C:/avenguard/stored_plates/cropped.jpg"
                #original="C:/avenguard/stored_plates/original.jpg"
                
                for plate in results['results']:
                    img = cv2.imread(nows) 
                    original="C:/avenguard/stored_plates/"+snow+"_original.jpg"
                    cv2.imwrite(original,img)
                    x1,x2,y1,y2=[plate['coordinates'][0]['x'], plate['coordinates'][2]['x'],plate['coordinates'][0]['y'],plate['coordinates'][2]['y']]
                    #img = cv2.imread(support)
                    crop_img = img[y1:y2, x1:x2]
                    cropped="C:/avenguard/stored_plates/"+snow+"_cropped.jpg"
                    
                    cv2.imwrite(cropped,crop_img)
                    #templates_path=r'C:\avenguard\Resources'
                    #for template_source in glob.glob(templates_path + "/*.jpg"):
                    # load the image image, convert it to grayscale, and detect edges
                    #template = cv2.imread(template_source)
                    #template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
                    #template = cv2.Canny(template, 50, 200)
                    #(tH, tW) = template.shape[:2]
    
                        #Type=getLicenceType(template,cropped)
                        
                    #if Type and Type=='SP':
                        #plate['plate']="RP "+plate['plate']
                        # break
                    if  (plate['plate'].isdigit()):
                        plate['plate']=f.insert_type(plate['plate'])
                
                    called=r"C:/avenguard/callable_plates/"+plate['plate']+".jpg"
                    cv2.imwrite(called,crop_img)
                # open a file for writing
                results_data = open(r'C:\avenguard\files\results2.csv', 'a')

                # create the csv writer object

                csvwriter = csv.writer(results_data, delimiter =',',quoting=csv.QUOTE_MINIMAL)

                count = 0

                for res in res_data:

                      if count == 0:

                             header = res.keys()

                             #csvwriter.writerow(header)

                             count += 1

                      csvwriter.writerow(res.values())

                results_data.close()     
                try:
                    
                    v = Image.open(cropped).resize((166,35), Image.ANTIALIAS)
                    logo = ImageTk.PhotoImage(v)
                    text="the detected license : "
                    w1 = tk.Label(self, image=logo)
                    w1.image=logo #we have to keep a reference otherwise the image will be transparent totally transparent!!
                    w2 = tk.Label(self, 
                          justify=tk.LEFT,
                          padx = 10, 
                          text=text)
                    w2.grid(row = 6, column = 10, sticky="w")
                    w1.grid(row = 7, column = 10,padx = 10)
                    v2 = Image.open(original).resize((166,140), Image.ANTIALIAS)
                    logo2 = ImageTk.PhotoImage(v2)
                    text="the original image :                            "
                    w3 = tk.Label(self, image=logo2)
                    w3.image=logo2 #we have to keep a refrence otherwise the image will be transparent totally transparet!!
                    w4 = tk.Label(self, 
                          justify=tk.LEFT,
                          padx = 10, 
                          text=text)
                    w4.grid(row = 0, column = 10, sticky="w")
                    w3.grid(row = 1, column = 10,rowspan=4,sticky="nswe",padx = 10)
                except:
                    #print ("no plate to show")
                    original="C:/avenguard/stored_plates/small_no_plate.jpg"
                    v2 = Image.open(original)#.resize((166,140), Image.ANTIALIAS)
                    logo2 = ImageTk.PhotoImage(v2)
                    text="No licence to show keep waiting : "
                    w3 = tk.Label(self, image=logo2)
                    w3.image=logo2 #we have to keep a refrence otherwise the image will be transparent totally transparet!!
                    w4 = tk.Label(self, 
                          justify=tk.LEFT,
                          padx = 10, 
                          text=text)
                    w4.grid(row = 0, column = 10, sticky="w")
                    w3.grid(row = 1, column = 10,rowspan=4,sticky="nswe",padx = 10)
                list = os.listdir(dir)
                if len(list)==100:
                    for fl in list:
                        file_path = os.path.join(dir, fl)
                        try:
                            if os.path.isfile(file_path):
                                os.unlink(file_path)
                            #elif os.path.isdir(file_path): shutil.rmtree(file_path)
                        except Exception as e:
                            print(e)
                self.after(100,lambda:execute())
 
       
        tk.Frame.__init__(self, parent)
        
        self.controller = controller
    
        #the popup function is used to create a sepreate window for downloading the results of a given date   
        def popup_date():
            #first, we start by creatng the entry widget to get the date
            def get_date():
                affichage['text'] = reponse.get()	# lecture du contenu du widget "reponse"
                b = ttk.Button(win, text ='download file',command=download_file)
                b.pack()
                return affichage['text']
            #once we get the date, a save as window appears to select the destination of the new file.
            #we have, therefore to name our future file
            def download_file():
                from tkinter import filedialog
                import requests
                #here we are going to nstatantiate the save as window 
                self.filename =  filedialog.asksaveasfilename(initialdir = "/",title = "Select file",filetypes = (("csv files","*.csv"),("Excel files","*.xls"),("all files","*.*")))
                #reading the entred date and storing it
                selected_date=affichage['text']
                dir=self.filename+".csv"
            
                #data = requests.get('https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=MSFT&apikey=demo&datatype=csv')
                #with open(dir, 'w') as f:
                 #   writer = csv.writer(f)
                  #  reader = csv.reader(data.text.splitlines())

                   # for row in reader:
                    #    writer.writerow(row)
                #Load the table from the result file wghich the timestamp colomun corresponded to the given date
                table1 = etl.fromcsv(r'C:\avenguard\files\results.csv')
                table2 = etl.rowlenselect(table1, 12)
                # Alter the colums
                table2 = etl.cut(table2, 'plate','timestamp')
                
                #table2 = etl.tail(table2, 15)
                table2 = etl.select(table2,
                        lambda rec: rec.timestamp.split("_")[0] == selected_date )    
                # Save to new file in format xmls (excel file)
                #etl.tocsv(table2, dir)
                etl.toxlsx(table2, self.filename+'.xlsx')
                #once the download is achived we have to destroy the download window
                win.destroy()
            win = tk.Toplevel()
            win.wm_title("Download file")
            win.pack_propagate(False)
            win.configure(height=170,width=300)
            nom = tk.Label(win, text = 'Enter the desired date please :')
            reponse = tk.Entry(win)
            valeur = ttk.Button(win, text ='confirm',command=get_date)
            affichage = tk.Label(win)
            votre_add=tk.Label(win, text='you have selected the following date :')
            nom.pack(expand=True)
            reponse.pack(expand=True)
            valeur.pack(expand=True)
            votre_add.pack(expand=True)
            affichage.pack(expand=True)
        
        #button3.grid(row = 3, column = 11,rowspan=1)
        #button2.grid(row = 2, column = 11,rowspan=1)
        #button1.grid(row = 1, column = 11,rowspan=1)
            
        menubar = tk.Menu(self)

        # create a pulldown menu, and add it to the menu bar
        filemenu = tk.Menu(menubar, tearoff=0)
        
        filemenu.add_command(label="Run", command=execute)
        filemenu.add_command(label="show stream", command=f.combine_funcs(lambda: controller.show_frame("PageTwo"),show_stream))
        filemenu.add_command(label="save to excel", command=popup_date)
        filemenu.add_separator()
        filemenu.add_command(label="Go to the start page", command=lambda: controller.show_frame("StartPage"))
        filemenu.add_separator()
        filemenu.add_command(label="Exit", command=self.destroy)
        menubar.add_cascade(label="File", menu=filemenu)

        # create more pulldown menus
        settingmenu = tk.Menu(menubar, tearoff=0)
        settingmenu.add_command(label="select a camera", command=lambda: controller.show_frame("PageOne"))
        settingmenu.add_command(label="initialise environnement", command=lambda: c.get_table(f.transform_data(pd.read_csv(r'C:\avenguard\files\results.csv'))))
        menubar.add_cascade(label="Setting", menu=settingmenu)

        helpmenu = tk.Menu(menubar, tearoff=0)
        helpmenu.add_command(label="About", command=lambda: controller.show_frame("PageHelp"))
        menubar.add_cascade(label="Help", menu=helpmenu)

        # display the menu
        controller.config(menu=menubar)
        
class PageHelp(tk.Frame):

            
    def __init__(self, parent, controller):
            
        tk.Frame.__init__(self, parent)
        self.controller = controller
        
        button = ttk.Button(self, text="Go to the start page",
                           command=lambda: controller.show_frame("StartPage"))
        button.pack(side="bottom")
        
        label1 = tk.Label(self, text="Welcome to the ALPR solution developped By Avensys", font=("Helvetica", 16))
        label1.pack(side="top", fill="x", pady=4) 
        #avenguard="You are using Avenguard, the automatic license plate developped by Avensys.\nto start using this software go to the file menu and select the address IP to be\nlinked to the application. Once confirmed go to the file menu and press\ninitialise environnement."
        avenguard="""You are using Avenguard: the automatic license plate recognition (ALPR) developed 
by Avensys. To start using this software, go to the setting menu and select a valid IP
address to be linked with the ALPR the application. Once confirmed, go to the setting 
menu and press initialise environnement. After the success of initialization, you could
run the app using the run command under the file menu."""

        label2 = tk.Label(self, text=avenguard, font=("Helvetica", 11),justify='left',width=70)
        label2.pack(side="right", fill="x", anchor='e', pady=4) 

 
        
if __name__ == "__main__":
    app = SampleApp()
    app.mainloop()


ModuleNotFoundError: ignored

In [0]:
from openalpr import Alpr
from os import environ
alpr_dir=r'openalpr\openalpr_64'
environ['PATH'] = alpr_dir + ';' + environ['PATH']
alpr = Alpr('eu', alpr_dir + '/openalpr.conf', alpr_dir + '/runtime_data')
     
if not alpr.is_loaded():
    print("Error loading OpenALPR")
    sys.exit(1)
    
alpr.set_top_n(10)
alpr.set_default_region("kr2")
support="7bis.jpg"
car_model='test'
results={}
try:
    results = alpr.recognize_file(support)
except:
    print('Error recongnizing the image')
                
try:
    if results:
        t=results['results'][0]
        t[u'timestamp']="snow"
        t[u'car_model']=car_model
        res_data = results['results']
except:
    t=results['results']
results['results']

In [0]:
import csv
results_data = open(r'C:\avenguard\files\results2.csv', 'a')
csvwriter = csv.writer(results_data, delimiter =',',quoting=csv.QUOTE_MINIMAL)

count = 0

for res in res_data:

    if count == 0:

        #header = res.keys()

        #csvwriter.writerow(header)

        count += 1

        csvwriter.writerow(res.values())

results_data.close()

# Author: Miguel Martinez Lopez
# Version: 0.20

try:
    from Tkinter import Frame, Label, Message, StringVar, Canvas
    from ttk import Scrollbar
    from Tkconstants import *
except ImportError:
    from tkinter import Frame, Label, Message, StringVar, Canvas
    from tkinter.ttk import Scrollbar
    from tkinter.constants import *

import platform
from PIL import Image, ImageTk
OS = platform.system()

class Mousewheel_Support(object):    

    # implemetation of singleton pattern
    _instance = None
    def __new__(cls, *args, **kwargs):
        if not cls._instance:
            cls._instance = object.__new__(cls)
        return cls._instance

    def __init__(self, root, horizontal_factor = 2, vertical_factor=2):
        
        self._active_area = None
        
        if isinstance(horizontal_factor, int):
            self.horizontal_factor = horizontal_factor
        else:
            raise Exception("Vertical factor must be an integer.")

        if isinstance(vertical_factor, int):
            self.vertical_factor = vertical_factor
        else:
            raise Exception("Horizontal factor must be an integer.")

        if OS == "Linux" :
            root.bind_all('<4>', self._on_mousewheel,  add='+')
            root.bind_all('<5>', self._on_mousewheel,  add='+')
        else:
            # Windows and MacOS
            root.bind_all("<MouseWheel>", self._on_mousewheel,  add='+')

    def _on_mousewheel(self,event):
        if self._active_area:
            self._active_area.onMouseWheel(event)

    def _mousewheel_bind(self, widget):
        self._active_area = widget

    def _mousewheel_unbind(self):
        self._active_area = None

    def add_support_to(self, widget=None, xscrollbar=None, yscrollbar=None, what="units", horizontal_factor=None, vertical_factor=None):
        if xscrollbar is None and yscrollbar is None:
            return

        if xscrollbar is not None:
            horizontal_factor = horizontal_factor or self.horizontal_factor

            xscrollbar.onMouseWheel = self._make_mouse_wheel_handler(widget,'x', self.horizontal_factor, what)
            xscrollbar.bind('<Enter>', lambda event, scrollbar=xscrollbar: self._mousewheel_bind(scrollbar) )
            xscrollbar.bind('<Leave>', lambda event: self._mousewheel_unbind())

        if yscrollbar is not None:
            vertical_factor = vertical_factor or self.vertical_factor

            yscrollbar.onMouseWheel = self._make_mouse_wheel_handler(widget,'y', self.vertical_factor, what)
            yscrollbar.bind('<Enter>', lambda event, scrollbar=yscrollbar: self._mousewheel_bind(scrollbar) )
            yscrollbar.bind('<Leave>', lambda event: self._mousewheel_unbind())

        main_scrollbar = yscrollbar if yscrollbar is not None else xscrollbar
        
        if widget is not None:
            if isinstance(widget, list) or isinstance(widget, tuple):
                list_of_widgets = widget
                for widget in list_of_widgets:
                    widget.bind('<Enter>',lambda event: self._mousewheel_bind(widget))
                    widget.bind('<Leave>', lambda event: self._mousewheel_unbind())

                    widget.onMouseWheel = main_scrollbar.onMouseWheel
            else:
                widget.bind('<Enter>',lambda event: self._mousewheel_bind(widget))
                widget.bind('<Leave>', lambda event: self._mousewheel_unbind())

                widget.onMouseWheel = main_scrollbar.onMouseWheel

    @staticmethod
    def _make_mouse_wheel_handler(widget, orient, factor = 1, what="units"):
        view_command = getattr(widget, orient+'view')
        
        if OS == 'Linux':
            def onMouseWheel(event):
                if event.num == 4:
                    view_command("scroll",(-1)*factor, what)
                elif event.num == 5:
                    view_command("scroll",factor, what) 
                
        elif OS == 'Windows':
            def onMouseWheel(event):        
                view_command("scroll",(-1)*int((event.delta/120)*factor), what) 
        
        elif OS == 'Darwin':
            def onMouseWheel(event):        
                view_command("scroll",event.delta, what)
        
        return onMouseWheel

class Scrolling_Area(Frame, object):

    def __init__(self, master, width=None, anchor=N, height=None, mousewheel_speed = 2, scroll_horizontally=True, xscrollbar=None, scroll_vertically=True, yscrollbar=None, outer_background=None, inner_frame=Frame, **kw):
        Frame.__init__(self, master, class_=self.__class__)

        if outer_background:
            self.configure(background=outer_background)

        self.grid_columnconfigure(0, weight=1)
        self.grid_rowconfigure(0, weight=1)
        
        self._width = width
        self._height = height

        self.canvas = Canvas(self, background=outer_background, highlightthickness=0, width=width, height=height)
        self.canvas.grid(row=0, column=0, sticky=N+E+W+S)

        if scroll_vertically:
            if yscrollbar is not None:
                self.yscrollbar = yscrollbar
            else:
                self.yscrollbar = Scrollbar(self, orient=VERTICAL)
                self.yscrollbar.grid(row=0, column=1,sticky=N+S)
        
            self.canvas.configure(yscrollcommand=self.yscrollbar.set)
            self.yscrollbar['command']=self.canvas.yview
        else:
            self.yscrollbar = None

        if scroll_horizontally:
            if xscrollbar is not None:
                self.xscrollbar = xscrollbar
            else:
                self.xscrollbar = Scrollbar(self, orient=HORIZONTAL)
                self.xscrollbar.grid(row=1, column=0, sticky=E+W)
            
            self.canvas.configure(xscrollcommand=self.xscrollbar.set)
            self.xscrollbar['command']=self.canvas.xview
        else:
            self.xscrollbar = None

        self.rowconfigure(0, weight=1)
        self.columnconfigure(0, weight=1)
        
        self.innerframe = inner_frame(self.canvas, **kw)
        self.innerframe.pack(anchor=anchor)
        
        self.canvas.create_window(0, 0, window=self.innerframe, anchor='nw', tags="inner_frame")

        self.canvas.bind('<Configure>', self._on_canvas_configure)

        Mousewheel_Support(self).add_support_to(self.canvas, xscrollbar=self.xscrollbar, yscrollbar=self.yscrollbar)

    @property
    def width(self):
        return self.canvas.winfo_width()

    @width.setter
    def width(self, width):
        self.canvas.configure(width= width)

    @property
    def height(self):
        return self.canvas.winfo_height()
        
    @height.setter
    def height(self, height):
        self.canvas.configure(height = height)
        
    def set_size(self, width, height):
        self.canvas.configure(width=width, height = height)

    def _on_canvas_configure(self, event):
        width = max(self.innerframe.winfo_reqwidth(), event.width)
        height = max(self.innerframe.winfo_reqheight(), event.height)

        self.canvas.configure(scrollregion="0 0 %s %s" % (width, height))
        self.canvas.itemconfigure("inner_frame", width=width, height=height)

    def update_viewport(self):
        self.update()

        window_width = self.innerframe.winfo_reqwidth()
        window_height = self.innerframe.winfo_reqheight()
        
        if self._width is None:
            canvas_width = window_width
        else:
            canvas_width = min(self._width, window_width)
            
        if self._height is None:
            canvas_height = window_height
        else:
            canvas_height = min(self._height, window_height)

        self.canvas.configure(scrollregion="0 0 %s %s" % (window_width, window_height), width=canvas_width, height=canvas_height)
        self.canvas.itemconfigure("inner_frame", width=window_width, height=window_height)

class Cell(Frame):
    """Base class for cells"""

class Data_Cell(Cell):
    def __init__(self, master, variable, anchor=W, bordercolor=None, borderwidth=1, padx=0, pady=0, background=None, foreground=None, font=None):
        Cell.__init__(self, master, background=background, highlightbackground=bordercolor, highlightcolor=bordercolor, highlightthickness=borderwidth, bd= 0)

        self._message_widget = Message(self, textvariable=variable, font=font, background=background, foreground=foreground)
        self._message_widget.pack(expand=True, padx=padx, pady=pady, anchor=anchor)

class Header_Cell(Cell):
    def __init__(self, master, text, bordercolor=None, borderwidth=1, padx=0, pady=0, background=None, foreground=None, font=None, anchor=CENTER, separator=True):
        Cell.__init__(self, master, background=background, highlightbackground=bordercolor, highlightcolor=bordercolor, highlightthickness=borderwidth, bd= 0)
        self.pack_propagate(False)

        self._header_label = Label(self, text=text, background=background, foreground=foreground, font=font)
        self._header_label.pack(padx=padx, pady=pady, expand=True)

        if separator and bordercolor is not None:
            separator = Frame(self, height=2, background=bordercolor, bd=0, highlightthickness=0, class_="Separator")
            separator.pack(fill=X, anchor=anchor)

        self.update()
        height = self._header_label.winfo_reqheight() + 2*padx
        width = self._header_label.winfo_reqwidth() + 2*pady

        self.configure(height=height, width=width)
        
class Table(Frame):
    def __init__(self, master, columns, column_weights=None, column_minwidths=None, height=500, minwidth=20, minheight=20, padx=5, pady=5, cell_font=None, cell_foreground="black", cell_background="white", cell_anchor=W, header_font=None, header_background="white", header_foreground="black", header_anchor=CENTER, bordercolor = "#999999", innerborder=True, outerborder=True, stripped_rows=("#EEEEEE", "white"), on_change_data=None, mousewheel_speed = 2, scroll_horizontally=False, scroll_vertically=True):
        outerborder_width = 1 if outerborder else 0

        Frame.__init__(self,master, bd= 0)

        self._cell_background = cell_background
        self._cell_foreground = cell_foreground
        self._cell_font = cell_font
        self._cell_anchor = cell_anchor
        
        self._stripped_rows = stripped_rows

        self._padx = padx
        self._pady = pady
        
        self._bordercolor = bordercolor
        self._innerborder_width = 1 if innerborder else 0

        self._data_vars = []

        self._columns = columns
        
        self._number_of_rows = 0
        self._number_of_columns = len(columns)

        self.grid_columnconfigure(0, weight=1)
        self.grid_rowconfigure(1, weight=1)
        
        self._head = Frame(self, highlightbackground=bordercolor, highlightcolor=bordercolor, highlightthickness=outerborder_width, bd= 0)
        self._head.grid(row=0, column=0, sticky=E+W)

        header_separator = False if outerborder else True

        for j in range(len(columns)):
            column_name = columns[j]

            header_cell = Header_Cell(self._head, text=column_name, borderwidth=self._innerborder_width, font=header_font, background=header_background, foreground=header_foreground, padx=padx, pady=pady, bordercolor=bordercolor, anchor=header_anchor, separator=header_separator)
            header_cell.grid(row=0, column=j, sticky=N+E+W+S)

        add_scrollbars = scroll_horizontally or scroll_vertically
        if add_scrollbars:
            if scroll_horizontally:
                xscrollbar = Scrollbar(self, orient=HORIZONTAL)
                xscrollbar.grid(row=2, column=0, sticky=E+W)
            else:
                xscrollbar = None

            if scroll_vertically:
                yscrollbar = Scrollbar(self, orient=VERTICAL)
                yscrollbar.grid(row=1, column=1, sticky=N+S)
            else:
                yscrollbar = None

            scrolling_area = Scrolling_Area(self, width=self._head.winfo_reqwidth(), height=height, scroll_horizontally=scroll_horizontally, xscrollbar=xscrollbar, scroll_vertically=scroll_vertically, yscrollbar=yscrollbar)
            scrolling_area.grid(row=1, column=0, sticky=E+W)

            self._body = Frame(scrolling_area.innerframe, highlightbackground=bordercolor, highlightcolor=bordercolor, highlightthickness=outerborder_width, bd= 0)
            self._body.pack()
            
            def on_change_data():
                scrolling_area.update_viewport()

        else:
            self._body = Frame(self, height=height, highlightbackground=bordercolor, highlightcolor=bordercolor, highlightthickness=outerborder_width, bd= 0)
            self._body.grid(row=1, column=0, sticky=N+E+W+S)

        if column_weights is None:
            for j in range(len(columns)):
                self._body.grid_columnconfigure(j, weight=1)
        else:
            for j, weight in enumerate(column_weights):
                self._body.grid_columnconfigure(j, weight=weight)

        if column_minwidths is not None:
            for j, minwidth in enumerate(column_minwidths):
                if minwidth is None:
                    header_cell = self._head.grid_slaves(row=0, column=j)[0]
                    minwidth = header_cell.winfo_reqwidth()

                self._body.grid_columnconfigure(j, minsize=minwidth)
        else:
            for j in range(len(columns)):
                header_cell = self._head.grid_slaves(row=0, column=j)[0]
                minwidth = header_cell.winfo_reqwidth()

                self._body.grid_columnconfigure(j, minsize=minwidth)

        self._on_change_data = on_change_data

    def _append_n_rows(self, n):
        number_of_rows = self._number_of_rows
        number_of_columns = self._number_of_columns

        for i in range(number_of_rows, number_of_rows+n):
            list_of_vars = []
            for j in range(number_of_columns):
                var = StringVar()
                list_of_vars.append(var)

                if self._stripped_rows:
                    cell = Data_Cell(self._body, borderwidth=self._innerborder_width, variable=var, bordercolor=self._bordercolor, padx=self._padx, pady=self._pady, background=self._stripped_rows[i%2], foreground=self._cell_foreground, font=self._cell_font, anchor=self._cell_anchor)
                else:
                    cell = Data_Cell(self._body, borderwidth=self._innerborder_width, variable=var, bordercolor=self._bordercolor, padx=self._padx, pady=self._pady, background=self._cell_background, foreground=self._cell_foreground, font=self._cell_font, anchor=self._cell_anchor)

                cell.grid(row=i, column=j, sticky=N+E+W+S)

            self._data_vars.append(list_of_vars)
            
        if number_of_rows == 0:
            for j in range(self.number_of_columns):
                header_cell = self._head.grid_slaves(row=0, column=j)[0]
                data_cell = self._body.grid_slaves(row=0, column=j)[0]
                data_cell.bind("<Configure>", lambda event, header_cell=header_cell: header_cell.configure(width=event.width), add="+")

        self._number_of_rows += n

    def _pop_n_rows(self, n):
        number_of_rows = self._number_of_rows
        number_of_columns = self._number_of_columns
        
        for i in range(number_of_rows-n, number_of_rows):
            for j in range(number_of_columns):
                self._body.grid_slaves(row=i, column=j)[0].destroy()
            
            self._data_vars.pop()
    
        self._number_of_rows -= n

    def set_data(self, data):
        n = len(data)
        m = len(data[0])

        number_of_rows = self._number_of_rows

        if number_of_rows > n:
            self._pop_n_rows(number_of_rows-n)
        elif number_of_rows < n:
            self._append_n_rows(n-number_of_rows)

        for i in range(n):
            for j in range(m):
                self._data_vars[i][j].set(data[i][j])

        if self._on_change_data is not None: self._on_change_data()

    def get_data(self):
        number_of_rows = self._number_of_rows
        number_of_columns = self.number_of_columns
        
        data = []
        for i in range(number_of_rows):
            row = []
            row_of_vars = self._data_vars[i]
            for j in range(number_of_columns):
                cell_data = row_of_vars[j].get()
                row.append(cell_data)
            
            data.append(row)
        return data

    @property
    def number_of_rows(self):
        return self._number_of_rows

    @property
    def number_of_columns(self):
        return self._number_of_columns

    def row(self, index, data=None):
        if data is None:
            row = []
            row_of_vars = self._data_vars[index]

            for j in range(self.number_of_columns):
                row.append(row_of_vars[j].get())
                
            return row
        else:
            number_of_columns = self.number_of_columns
            
            if len(data) != number_of_columns:
                raise ValueError("data has no %d elements: %s"%(number_of_columns, data))

            row_of_vars = self._data_vars[index]
            for j in range(number_of_columns):
                row_of_vars[index][j].set(data[j])
                
            if self._on_change_data is not None: self._on_change_data()

    def column(self, index, data=None):
        number_of_rows = self._number_of_rows

        if data is None:
            column= []

            for i in range(number_of_rows):
                column.append(self._data_vars[i][index].get())
                
            return column
        else:            
            if len(data) != number_of_rows:
                raise ValueError("data has no %d elements: %s"%(number_of_rows, data))

            for i in range(number_of_columns):
                self._data_vars[i][index].set(data[i])

            if self._on_change_data is not None: self._on_change_data()

    def clear(self):
        number_of_rows = self._number_of_rows
        number_of_columns = self._number_of_columns

        for i in range(number_of_rows):
            for j in range(number_of_columns):
                self._data_vars[i][j].set("")

        if self._on_change_data is not None: self._on_change_data()

    def delete_row(self, index):
        i = index
        while i < self._number_of_rows:
            row_of_vars_1 = self._data_vars[i]
            row_of_vars_2 = self._data_vars[i+1]

            j = 0
            while j <self.number_of_columns:
                row_of_vars_1[j].set(row_of_vars_2[j])

            i += 1

        self._pop_n_rows(1)

        if self._on_change_data is not None: self._on_change_data()

    def insert_row(self, data, index=END):
        self._append_n_rows(1)

        if index == END:
            index = self._number_of_rows - 1
        
        i = self._number_of_rows-1
        while i > index:
            row_of_vars_1 = self._data_vars[i-1]
            
            row_of_vars_2 = self._data_vars[i]
            
            j = 0
            while j < self.number_of_columns:
                row_of_vars_2[j].set(row_of_vars_1[j])
                j += 1
            i -= 1

        list_of_cell_vars = self._data_vars[index]
        for cell_var, cell_data in zip(list_of_cell_vars, data):
            cell_var.set(cell_data)

        if self._on_change_data is not None: self._on_change_data()

    def cell(self, row, column, data=None):
        """Get the value of a table cell"""
        if data is None:
            return self._data_vars[row][column].get()
        else:
            self._data_vars[row][column].set(data)
            if self._on_change_data is not None: self._on_change_data()

    def __getitem__(self, index):
        if isinstance(index, tuple):
            row, column = index
            return self.cell(row, column)
        else:
            raise Exception("Row and column indices are required")
        
    def __setitem__(self, index, value):
        if isinstance(index, tuple):
            row, column = index
            
            self.cell(row, column, label)
        else:
            raise Exception("Row and column indices are required")

    def on_change_data(self, callback):
        self._on_change_data = callback

if __name__ == "__main__":
    try:
        from Tkinter import Tk
    except ImportError:
        from tkinter import Tk
    import petl as etl
    #Load the table
    table1 = etl.fromcsv(r'C:\avenguard\files\results.csv')
    table2 = etl.rowlenselect(table1, 12)
    
    table2 = etl.cut(table2, 'plate','timestamp','region')
    table2 = etl.split(table2, 'timestamp', '_', ['date', 'heure'])
    #table2 = etl.split(table2, 'date', '-', ['date', 'heure'])
    headers=etl.header(table2)
    root = Tk()

    table = Table(root, headers, column_minwidths=[150,150,150,150])
    table.pack(padx=10,pady=10)

    #table.set_data([[1,2,3],[4,5,6], [7,8,9], [10,11,12], [13,14,15],[15,16,18], [19,20,21]])
    #table.cell(0,0, " a fdas fasd fasdf asdf asdfasdf asdf asdfa sdfas asd sadf ")
    for row in range(0,len(table2)):
        table.insert_row(table2[row])
    #table.insert_row([25,26,27])
    
    root.update()
    root.geometry("%sx%s"%(root.winfo_reqwidth(),250))

    root.mainloop()

In [0]:
import petl as etl
from PIL import Image, ImageTk
import functions as f
#Load the table
table1 = etl.fromcsv(r'C:\avenguard\files\results.csv')
table2 = etl.rowlenselect(table1, 12)
    
table2 = etl.cut(table2, 'plate','timestamp','region')
table2 = etl.split(table2, 'timestamp', '_', ['date', 'heure'])
table2 = etl.tail(table2,15)
#headers=etl.header(table2)
import tkinter as tk

class ExampleApp(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        t = SimpleTable(self, 10,2)
        t.pack(side="top", fill="x")
        #t.set(0,0,"Hello, world")

class SimpleTable(tk.Frame):
    def __init__(self, parent, rows=10, columns=2):
        # use black background so it "peeks through" to 
        # form grid lines
        tk.Frame.__init__(self, parent, background="black")
        self._widgets = []
        canvas = tk.Canvas(width=300, height=300, bg='white')   
        #canvas.pack(expand='YES', fill='both') 
        hbar=tk.Scrollbar(canvas,orient='h')
        hbar.pack(side="bottom",fill="x")
        hbar.config(command=canvas.xview)
        vbar=tk.Scrollbar(canvas,orient="v")
        vbar.pack(side="right",fill="y")
        vbar.config(command=canvas.yview)
        canvas.config(width=300,height=300)
        canvas.config(xscrollcommand=hbar.set, yscrollcommand=vbar.set)

        
        r=0
        for column in table2:
            current_row = []
            c=0
            for row in column:
                if r==0:
                    if row=="plate":
                        label = tk.Label(self, text="plaque detecté",borderwidth=0, width=24,height=2,fg="blue")
                        label.grid(row=r, column=c, sticky="nsew", padx=1, pady=1)
                        current_row.append(label)
                        label2 = tk.Label(self, text='numéro detecté',borderwidth=0, width=24,height=2,fg="blue")
                        label2.grid(row=r, column=c+1, sticky="nsew", padx=1, pady=1)
                        current_row.append(label2)
                    else:
                        label = tk.Label(self, text=row,borderwidth=0, width=24,height=2,fg="blue")
                        label.grid(row=r, column=c+1, sticky="nsew", padx=1, pady=1)
                        current_row.append(label)
                elif c==0:
                    img=r"C:/avenguard/callable_plates/"+row+".jpg"
                    v=Image.open(img).resize((166,35),Image.ANTIALIAS)
                    logo=ImageTk.PhotoImage(v)
                    label=tk.Label(self,image=logo,text=row)
                    label.image=logo
                    label.grid(row=r, column=c, sticky="nsew", padx=1, pady=1)
                    current_row.append(label)
                    label2 = tk.Label(self, text=f.insert_type(row),borderwidth=0, width=24,height=2)
                    label2.grid(row=r, column=c+1, sticky="nsew", padx=1, pady=1)
                    current_row.append(label2)
                elif c==1:
                    label = tk.Label(self, text=row.replace("-",":"),borderwidth=0, width=24,height=2)
                    label.grid(row=r, column=c+1, sticky="nsew", padx=1, pady=1)
                    current_row.append(label)
                else:        
                    label = tk.Label(self, text=row,borderwidth=0, width=24,height=2)
                    label.grid(row=r, column=c+1, sticky="nsew", padx=1, pady=1)
                    current_row.append(label)
                c=c+1
            self._widgets.append(current_row)
            r=r+1
        #canvas.create_window((50,50),window=self)
        for column in range(columns):
            self.grid_columnconfigure(column, weight=1)


    def set(self, row, column, value):
        widget = self._widgets[row][column]
        widget.configure(text=value)

if __name__ == "__main__":
    app = ExampleApp()
    app.mainloop()

In [0]:
plt.figure()
sns.countplot(x='missile', hue='party', data=df, palette='RdBu')
plt.xticks([0,1], ['No', 'Yes'])
plt.show()